In [6]:
import pandas as pd
import numpy as np
import requests
import json
import time

In [2]:
#base_url = "https://api.prod.bs-aws-stage.com/search/cases?addressTypes=villa,condo,terraced+house,holiday+house,cooperative,farm,hobby+farm,full+year+plot,villa+apartment,holiday+plot&per_page=50&page=1&sortAscending=true&sortBy=timeOnMarket"

In [3]:
n_max = 45134
per_page = 500
pages = [x for x in range(1,int(n_max/per_page)+1,1)]

In [4]:
urls = [f"https://api.prod.bs-aws-stage.com/search/cases?addressTypes=villa,condo,terraced+house,holiday+house,cooperative,farm,hobby+farm,full+year+plot,villa+apartment,holiday+plot&per_page={per_page}&page={x}&sortAscending=true&sortBy=timeOnMarket" for x in pages]

In [7]:
data = pd.DataFrame()
for url in urls:
    response = requests.get(url[:21]) # at urls[21] there is an error, so we only request for the first 10,000 observations
    print(url)
    data_500 = pd.DataFrame(response.json()["cases"])
    df_2 = [data, data_500]
    data = pd.concat(df_2, ignore_index=True)
    time.sleep(5)

ConnectionError: HTTPSConnectionPool(host='api.prod.bs-a', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000132084043D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [8]:
data = pd.read_csv("data/boligsiden_10000.csv")

In [9]:
data.drop('_links', axis=1, inplace=True)

data.drop('caseID', axis=1, inplace=True)

data.drop('defaultImage', axis=1, inplace=True)

data.drop('images', axis=1, inplace=True)

data.drop('totalFavourites', axis=1, inplace=True)

data.drop('weightedArea', axis=1, inplace=True)

data.drop('distinction', axis=1, inplace=True)

data.drop('pageViews', axis=1, inplace=True)

data.drop('highlighted', axis=1, inplace=True)

data.drop('totalClickCount', axis=1, inplace=True)

data.drop('nextOpenHouse', axis=1, inplace=True)

data.drop('providerCaseID', axis=1, inplace=True)

data.drop('secondaryAddressType', axis=1, inplace=True)

data.drop('descriptionBody', axis=1, inplace=True)

data.drop('descriptionTitle', axis=1, inplace=True)

data.drop('slug', axis=1, inplace=True)

data.drop('status', axis=1, inplace=True)

data.drop('realtor', axis=1, inplace=True)

data.drop('cooperative', axis=1, inplace=True)

KeyError: "['_links'] not found in axis"

In [10]:
# extracting information on number of buildings from the "address" variable
list_numBuildings = []
for i in range(0,len(data),1):
    try:
        list_numBuildings.append(len(data["address"][i]["buildings"]))
    except:
        list_numBuildings.append(0)

data["numberOfBuildings"] = list_numBuildings

In [11]:
# if number of buildings is 0, it gets set to 1, so that e.g. 1 appartment counts as 1 building
for i in range(0,len(data),1):
    if (data["numberOfBuildings"].iloc[i] == 0):
        data["numberOfBuildings"].iloc[i] = 1

C:\Users\MZB\AppData\Local\Temp/ipykernel_8880/308494572.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["numberOfBuildings"].iloc[i] = 1


In [12]:
# Extracting Zip Code and Municipality Information from the Address Variable

data["zipCode"] = [data.address[i]["zipCode"] for i in range(0,len(data),1)]

data["municipality"] = [data["address"][i]["municipality"] for i in range(0,len(data),1)]

data["municipalityCode"] = [data["municipality"][i]["municipalityCode"] for i in range(0,len(data),1)]

data["municipality"] = [data["municipality"][i]["name"] for i in range(0,len(data),1)]



AttributeError: 'DataFrame' object has no attribute 'address'

In [13]:
# for easy handling of the coordinates: two columns instead of one
data["latitude"] = [data["coordinates"][i]["lat"] for i in range(0,len(data),1)]
data["longitude"] = [data["coordinates"][i]["lon"] for i in range(0,len(data),1)]

KeyError: 'coordinates'

In [14]:
data.drop('coordinates', axis=1, inplace=True)

KeyError: "['coordinates'] not found in axis"

In [15]:
data.drop('address', axis=1, inplace=True)

KeyError: "['address'] not found in axis"

In [16]:
data.drop('realEstate', axis=1, inplace=True)

KeyError: "['realEstate'] not found in axis"

### Basement and Lot
New indicator variables for Basement and Lot, that indicate whether or not there is any basement or any lot included in the real estate unit 

Nan are replaced with 0 based on the assumption that if the real estate agency does not report any basement/lot, then there is none.

In [17]:
basement_list = []
for i in range(0,len(data),1):
    if (data["basementArea"][i] > 0):
        basement_list.append(1)
    else:
        basement_list.append(0)
data["basement"] = basement_list

In [18]:
basement_list = []
for i in range(0,len(data),1):
    if (data["basementArea"][i] > 0):
        basement_list.append(data["basementArea"][i])
    else:
        basement_list.append(0)
data["basementArea"] = basement_list

In [19]:
lot_list = []
for i in range(0,len(data),1):
    if (data["lotArea"][i] > 0):
        lot_list.append(1)
    else:
        lot_list.append(0)
data["lot"] = lot_list

In [20]:
lot_list = []
for i in range(0,len(data),1):
    if (data["lotArea"][i] > 0):
        lot_list.append(data["lotArea"][i])
    else:
        lot_list.append(0)
data["lotArea"] = lot_list

In [21]:
data.columns

Index(['Unnamed: 0', 'addressType', 'basementArea', 'caseUrl', 'energyLabel',
       'housingArea', 'lotArea', 'monthlyExpense', 'numberOfFloors',
       'numberOfRooms', 'perAreaPrice', 'priceCash', 'priceChangePercentage',
       'yearBuilt', 'municipalityCode', 'numberOfBuildings', 'zipCode',
       'municipality', 'latitude', 'longitude', 'basement', 'lot'],
      dtype='object')

In [22]:
data

,Unnamed: 0,addressType,basementArea,caseUrl,energyLabel,housingArea,lotArea,monthlyExpense,numberOfFloors,numberOfRooms,...,priceChangePercentage,yearBuilt,municipalityCode,numberOfBuildings,zipCode,municipality,latitude,longitude,basement,lot
0,0,villa,60.0,https://www.brikk.dk/ejendom/industrivej-2-662...,d,210.0,1233.0,1718,2.0,6.0,...,0.00,1968.0,575,1,6622,Vejen,55.571290,9.146651,1,1
1,1,farm,0.0,https://www.estaldo.com/listing/22fa0be5-dd95-...,e,472.0,84093.0,2334,1.0,4.0,...,0.00,1936.0,860,1,9800,Hjørring,57.416790,9.964815,0,1
2,2,villa,127.0,https://www.brikk.dk/ejendom/storegade-33-6640...,e,124.0,614.0,2169,3.0,5.0,...,0.00,1945.0,621,1,6640,Kolding,55.479263,9.298074,1,1
3,3,villa,69.0,http://www.nybolig.dk/maegler/pages/property-p...,e,69.0,497.0,2415,2.0,2.0,...,0.00,1946.0,400,1,3700,Bornholm,55.109230,14.709151,1,1
4,4,villa,80.0,https://www.danbolig.dk?propertyid=0950000496&...,c,179.0,1078.0,3634,2.0,5.0,...,0.00,1961.0,630,1,7100,Vejle,55.694424,9.553193,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,villa,0.0,https://home.dk/sag/8010000572,c,143.0,1062.0,1658,2.0,4.0,...,0.00,1900.0,813,1,9900,Frederikshavn,57.473946,10.481430,0,1
9996,9996,villa,0.0,http://www.estate-maeglerne.dk/maegler/pages/p...,d,232.0,1214.0,6232,1.0,6.0,...,0.00,1961.0,190,1,3500,Furesø,55.756330,12.415566,0,1
9997,9997,villa,0.0,https://www.danbolig.dk?propertyid=0140000466&...,d,134.0,1027.0,1692,2.0,3.0,...,0.00,1947.0,316,1,4340,Holbæk,55.593227,11.750100,0,1
9998,9998,condo,0.0,https://www.danbolig.dk?propertyid=2100000090&...,a2010,95.0,0.0,3085,5.0,3.0,...,-1.79,1989.0,101,1,2400,København,55.705082,12.531357,0,0


### There might be colinearity, for the variables lot, basement (and future indicator variables for addressType)

Theory: All units of type villa appartment have no basement and no lot

## Preview
Villas and Appartements in Copenhagen

In [ ]:
data[((data["addressType"] == "villa") | (data["addressType"] == "villa apartment")) & (data["municipality"] == "København") ]

In [ ]:
data.columns

In [ ]:
# Save data to csv

In [23]:
data.to_csv("data/boligsiden_2.csv")